<a href="https://colab.research.google.com/github/lenahdtr/fallstudie_lkws/blob/main/KI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie- LKW Zuteilung - KI


In [18]:
! pip install -q pyscipopt

In [19]:
import math
import pandas as pd
from datetime import datetime, time, timedelta
from pyscipopt import Model, quicksum

# Daten laden

*Daten aus den Tabellen, die hard codiert werden:*

In [20]:
DIESEL_TRUCKS = [
    {
        "truck_model": "ActrosL",
        "capex_yearly": 24000,
        "opex_yearly": 6000,
        "avg_diesel_per_100km": 26,
        "kfz_yearly": 556,
        "gross_vehicle_weight": 40,
        "emission_class": "EURO 6",
        "co2_emission_class": 1,
    }
]

In [21]:

ELECTRIC_TRUCKS = [
    {
        "truck_model": "eActros600",
        "capex_yearly": 60000,
        "opex_yearly": 6000,
        "avg_energy_kWh_per_100km": 110,
        "thg_yearly": 1000,
        "max_power": 400,
        "soc_max_kWh": 621,
    },
    {
        "truck_model": "eActros400",
        "capex_yearly": 50000,
        "opex_yearly": 5000,
        "avg_energy_kWh_per_100km": 105,
        "thg_yearly": 1000,
        "max_power": 400,
        "soc_max_kWh": 414,
    },
]

In [22]:
ROUTES = [
    {"route_id": "t-4", "route_name": "Nahverkehr", "distance_total": 250, "distance_toll": 150, "starttime": "06:45", "endtime": "17:15"},
    {"route_id": "t-5", "route_name": "Nahverkehr", "distance_total": 250, "distance_toll": 150, "starttime": "06:30", "endtime": "17:00"},
    {"route_id": "t-6", "route_name": "Nahverkehr", "distance_total": 250, "distance_toll": 150, "starttime": "06:00", "endtime": "16:30"},
    {"route_id": "s-1", "route_name": "Ditzingen", "distance_total": 120, "distance_toll": 32,  "starttime": "05:30", "endtime": "15:30"},
    {"route_id": "s-2", "route_name": "Ditzingen", "distance_total": 120, "distance_toll": 32,  "starttime": "06:00", "endtime": "16:00"},
    {"route_id": "s-3", "route_name": "Ditzingen", "distance_total": 120, "distance_toll": 32,  "starttime": "09:00", "endtime": "16:00"},
    {"route_id": "s-4", "route_name": "Ditzingen", "distance_total": 120, "distance_toll": 32,  "starttime": "06:30", "endtime": "16:30"},
    {"route_id": "w1",  "route_name": "Ditzingen", "distance_total": 100, "distance_toll": 32,  "starttime": "05:30", "endtime": "15:30"},
    {"route_id": "w2",  "route_name": "Ditzingen", "distance_total": 100, "distance_toll": 32,  "starttime": "08:00", "endtime": "18:00"},
    {"route_id": "w3",  "route_name": "Ditzingen", "distance_total": 100, "distance_toll": 32,  "starttime": "06:45", "endtime": "16:45"},
    {"route_id": "w4",  "route_name": "Ditzingen", "distance_total": 100, "distance_toll": 32,  "starttime": "06:00", "endtime": "16:00"},
    {"route_id": "w5",  "route_name": "Ditzingen", "distance_total": 100, "distance_toll": 32,  "starttime": "07:00", "endtime": "17:00"},
    {"route_id": "w6",  "route_name": "Ditzingen", "distance_total": 100, "distance_toll": 32,  "starttime": "05:30", "endtime": "15:30"},
    {"route_id": "w7",  "route_name": "Ditzingen", "distance_total": 100, "distance_toll": 32,  "starttime": "07:15", "endtime": "17:15"},
    {"route_id": "r1",  "route_name": "MultiStop", "distance_total": 285, "distance_toll": 259, "starttime": "18:00", "endtime": "22:30"},
    {"route_id": "r2",  "route_name": "MultiStop", "distance_total": 250, "distance_toll": 220, "starttime": "16:30", "endtime": "21:45"},
    {"route_id": "r3",  "route_name": "Schramberg", "distance_total": 235, "distance_toll": 219, "starttime": "17:45", "endtime": "21:30"},
    {"route_id": "h3",  "route_name": "Hettingen", "distance_total": 180, "distance_toll": 160, "starttime": "18:45", "endtime": "22:45"},
    {"route_id": "h4",  "route_name": "Hettingen", "distance_total": 180, "distance_toll": 160, "starttime": "18:30", "endtime": "22:30"},
    {"route_id": "k1",  "route_name": "Wendlingen am Neckar", "distance_total": 275, "distance_toll": 235, "starttime": "16:30", "endtime": "22:30"},
]


In [23]:
CHARGERS = [
    {"charger_model": "Alpitronic-50",  "capex_yearly": 3000,  "opex_yearly": 1000, "max_power": 50,  "charging_spots": 2},
    {"charger_model": "Alpitronic-200", "capex_yearly": 10000, "opex_yearly": 1500, "max_power": 200, "charging_spots": 2},
    {"charger_model": "Alpitronic-400", "capex_yearly": 16000, "opex_yearly": 2000, "max_power": 400, "charging_spots": 2},
]

Hilfsfunktionen

In [24]:
def time_to_minutes(t):
    # t is datetime.time
    return t.hour * 60 + t.minute

def build_time_steps(step_minutes=15):
    T = list(range(int(24*60/step_minutes)))  # 0..95 for 15min
    return T, step_minutes

def minutes_to_step(mins, step_minutes):
    return int(mins // step_minutes)

def overlap(a_start, a_end, b_start, b_end):
    # True if intervals overlap (strict overlap)
    return not (a_end <= b_start or b_end <= a_start)

# Globale Parameter



In [25]:
# -------------------------
# 2) Globale Parameter (hier als Inputs)
# -------------------------
DAYS_PER_YEAR = 260
STEP_MINUTES = 15
DT_HOURS = 0.25  # 15 min

# Kosten-/Preise (Inputs: ggf. anpassen)
ELECTRICITY_PRICE_EUR_PER_KWH = 0.25
ELECTRICITY_BASE_FEE_EUR_PER_YEAR = 1000
DEMAND_CHARGE_EUR_PER_KW_YEAR = 150   # <- falls in eurer Aufgabenstellung vorgegeben, hier setzen
DIESEL_PRICE_EUR_PER_L = 1.80         # <- Input
TOLL_EUR_PER_KM = 0.34

GRID_LIMIT_KW_BASE = 500
GRID_UPGRADE_KW = 500
GRID_UPGRADE_COST_EUR_PER_YEAR = 10000

# Speicherparameter
STORAGE_CAPEX_EUR_PER_KW_YEAR = 30
STORAGE_CAPEX_EUR_PER_KWH_YEAR = 350
STORAGE_OPEX_FRACTION = 0.02
ROUND_TRIP_EFF = 0.98
ETA_C = math.sqrt(ROUND_TRIP_EFF)
ETA_D = math.sqrt(ROUND_TRIP_EFF)
DOD = 0.975
MIN_SOC_FRAC = 1 - DOD  # 0.025


# Indexmengen

In [26]:

# -------------------------
# 3) Indexmengen
# -------------------------

routes = [r["route_id"] for r in ROUTES]
num_routes = len(routes)

# Truck-Slots: Obergrenze = Anzahl Routen (sicher)
trucks = list(range(1, num_routes + 1))

# Max 3 Ladesäulenplätze
charger_slots = [1, 2, 3]

# Ladepunkte je Säule (bei deinen Chargers jeweils 2)
plugs = [1, 2]

# 24h-Zeitraster
T = list(range(int(24 * 60 / STEP_MINUTES)))  # 0..95

diesel_model = DIESEL_TRUCKS[0]["truck_model"]
electric_models = [e["truck_model"] for e in ELECTRIC_TRUCKS]
vehicle_types = [diesel_model] + electric_models
electric_types = electric_models

charger_models = [c["charger_model"] for c in CHARGERS]

# Parameter-Mapping (Trucks/Chargers/Routen)

In [27]:
# -------------------------
# 4) Parameter-Mapping (Trucks/Chargers/Routen)
# -------------------------
truck_capex = {}
truck_opex = {}
truck_soc_max = {diesel_model: 0.0}
truck_pmax = {diesel_model: 0.0}
truck_thg = {diesel_model: 0.0}
truck_eRate = {diesel_model: 0.0}  # kWh/km

diesel_rate_l_per_km = DIESEL_TRUCKS[0]["avg_diesel_per_100km"] / 100.0
diesel_kfz_yearly = DIESEL_TRUCKS[0]["kfz_yearly"]

truck_capex[diesel_model] = float(DIESEL_TRUCKS[0]["capex_yearly"])
truck_opex[diesel_model] = float(DIESEL_TRUCKS[0]["opex_yearly"] + diesel_kfz_yearly)

for e in ELECTRIC_TRUCKS:
    v = e["truck_model"]
    truck_capex[v] = float(e["capex_yearly"])
    truck_opex[v] = float(e["opex_yearly"])
    truck_eRate[v] = float(e["avg_energy_kWh_per_100km"]) / 100.0
    truck_thg[v] = float(e["thg_yearly"])
    truck_pmax[v] = float(e["max_power"])
    truck_soc_max[v] = float(e["soc_max_kWh"])

charger_capex = {c["charger_model"]: float(c["capex_yearly"]) for c in CHARGERS}
charger_opex  = {c["charger_model"]: float(c["opex_yearly"])  for c in CHARGERS}
charger_pmax  = {c["charger_model"]: float(c["max_power"])    for c in CHARGERS}
charger_spots = {c["charger_model"]: int(c["charging_spots"]) for c in CHARGERS}

route_dist = {r["route_id"]: float(r["distance_total"]) for r in ROUTES}
route_toll = {r["route_id"]: float(r["distance_toll"]) for r in ROUTES}
route_start_str = {r["route_id"]: r["starttime"] for r in ROUTES}
route_end_str   = {r["route_id"]: r["endtime"] for r in ROUTES}

def time_str_to_minutes(hhmm: str) -> int:
    hh, mm = hhmm.split(":")
    return int(hh) * 60 + int(mm)

def minutes_to_step(mins: int) -> int:
    return int(mins // STEP_MINUTES)

def overlap(a_start, a_end, b_start, b_end) -> bool:
    return not (a_end <= b_start or b_end <= a_start)

route_start_min = {r: time_str_to_minutes(route_start_str[r]) for r in routes}
route_end_min   = {r: time_str_to_minutes(route_end_str[r])   for r in routes}

# Aktiv-Matrix active[r,t] = 1 wenn Route r in Zeitschritt t läuft
active = {(r, t): 0 for r in routes for t in T}
route_steps = {}
route_dist_per_step = {}

for r in routes:
    s_step = minutes_to_step(route_start_min[r])
    e_step = minutes_to_step(route_end_min[r])
    for t in T:
        if s_step <= t < e_step:
            active[(r, t)] = 1
    steps = max(1, e_step - s_step)
    route_steps[r] = steps
    route_dist_per_step[r] = route_dist[r] / steps

# Überlappungs-Paare (für "ein Truck kann nicht zwei parallele Routen fahren")
overlap_pairs = []
for i, r1 in enumerate(routes):
    for r2 in routes[i + 1:]:
        if overlap(route_start_min[r1], route_end_min[r1], route_start_min[r2], route_end_min[r2]):
            overlap_pairs.append((r1, r2))

route_start_step = {r: minutes_to_step(route_start_min[r]) for r in routes}


*Hilfsparameter - Big M*

In [28]:
M_charge = max(list(charger_pmax.values()) + list(truck_pmax.values()) + [0.0])
M_soc = max(truck_soc_max.values()) if len(truck_soc_max) > 0 else 0.0

# Initialisierung des Modells

In [29]:
model = Model("DepotElectrification")

# Entscheidungsvariablen

In [30]:
# Netz / Peak / Trafo / Speicher
gridImport = {t: model.addVar(lb=0.0, vtype="C", name=f"grid_import_kW_t{t}") for t in T}
gridPeak = model.addVar(lb=0.0, vtype="C", name="grid_peak_kW")
useTrafo = model.addVar(vtype="B", name="use_grid_upgrade")

storagePower = model.addVar(lb=0.0, vtype="C", name="storage_power_kW")
storageEnergy = model.addVar(lb=0.0, vtype="C", name="storage_energy_kWh")
storageSoc = {t: model.addVar(lb=0.0, vtype="C", name=f"storage_soc_kWh_t{t}") for t in T}
storageCharge = {t: model.addVar(lb=0.0, vtype="C", name=f"storage_charge_kW_t{t}") for t in T}
storageDischarge = {t: model.addVar(lb=0.0, vtype="C", name=f"storage_discharge_kW_t{t}") for t in T}

# Truck/Route/Type
truckType = {}
routeAssign = {}
routeAssignType = {}

# Charger install
chargerType = {}

# Charging & SOC
isPlugUsed = {}
chargePower = {}
soc = {}

for k in trucks:
    for v in vehicle_types:
        truckType[(k, v)] = model.addVar(vtype="B", name=f"truck_{k}_is_{v}")

    for r in routes:
        routeAssign[(k, r)] = model.addVar(vtype="B", name=f"truck_{k}_drives_route_{r}")
        for v in vehicle_types:
            routeAssignType[(k, r, v)] = model.addVar(vtype="B", name=f"truck_{k}_drives_route_{r}_as_{v}")

    for t in T:
        soc[(k, t)] = model.addVar(lb=0.0, vtype="C", name=f"soc_kWh_truck{k}_t{t}")
        for b in charger_slots:
            for l in plugs:
                isPlugUsed[(k, t, b, l)] = model.addVar(vtype="B", name=f"truck{k}_plugged_t{t}_charger{b}_plug{l}")
                chargePower[(k, t, b, l)] = model.addVar(lb=0.0, vtype="C", name=f"chargePower_kW_truck{k}_t{t}_charger{b}_plug{l}")

for b in charger_slots:
    for m in charger_models:
        chargerType[(b, m)] = model.addVar(vtype="B", name=f"charger{b}_is_{m}")

# Nebenbedingungen

In [31]:
# -------------------------
# 6) Nebenbedingungen (linear)
# -------------------------

# (1) Jeder Truck-Slot max 1 Typ
for k in trucks:
    model.addCons(quicksum(truckType[(k, v)] for v in vehicle_types) <= 1,
                  name=f"one_type_per_truck_{k}")

# (2) Route/Typ-Linearisierung
for k in trucks:
    for r in routes:
        model.addCons(routeAssign[(k, r)] == quicksum(routeAssignType[(k, r, v)] for v in vehicle_types),
                      name=f"assign_equals_sum_types_truck{k}_route{r}")
        for v in vehicle_types:
            model.addCons(routeAssignType[(k, r, v)] <= routeAssign[(k, r)],
                          name=f"type_assign_le_routeAssign_truck{k}_route{r}_{v}")
            model.addCons(routeAssignType[(k, r, v)] <= truckType[(k, v)],
                          name=f"type_assign_le_truckType_truck{k}_route{r}_{v}")

# (3) Jede Route genau einmal
for r in routes:
    model.addCons(quicksum(routeAssign[(k, r)] for k in trucks) == 1,
                  name=f"each_route_exactly_once_{r}")

# (4) Keine Überlappung je Truck
for k in trucks:
    for (r1, r2) in overlap_pairs:
        model.addCons(routeAssign[(k, r1)] + routeAssign[(k, r2)] <= 1,
                      name=f"no_overlap_truck{k}_{r1}_{r2}")

# (5) Ladesäulen: pro Slot max 1 Modell
for b in charger_slots:
    model.addCons(quicksum(chargerType[(b, m)] for m in charger_models) <= 1,
                  name=f"one_model_per_charger_slot_{b}")

# (6) Pro Plug max 1 Truck gleichzeitig
for t in T:
    for b in charger_slots:
        for l in plugs:
            model.addCons(quicksum(isPlugUsed[(k, t, b, l)] for k in trucks) <= 1,
                          name=f"one_truck_per_plug_t{t}_b{b}_l{l}")

# (7) Pro Truck pro Zeit max 1 Plug
for k in trucks:
    for t in T:
        model.addCons(quicksum(isPlugUsed[(k, t, b, l)] for b in charger_slots for l in plugs) <= 1,
                      name=f"truck_one_plug_at_time_k{k}_t{t}")

# (8) Leistung nur wenn Plug genutzt
for k in trucks:
    for t in T:
        for b in charger_slots:
            for l in plugs:
                model.addCons(chargePower[(k, t, b, l)] <= M_charge * isPlugUsed[(k, t, b, l)],
                              name=f"power_only_if_plugged_k{k}_t{t}_b{b}_l{l}")

# (9) Säulenleistungslimit je Säule
for t in T:
    for b in charger_slots:
        model.addCons(
            quicksum(chargePower[(k, t, b, l)] for k in trucks for l in plugs)
            <= quicksum(chargerType[(b, m)] * charger_pmax[m] for m in charger_models),
            name=f"charger_power_limit_t{t}_b{b}"
        )

# (10) Nur E-LKW dürfen laden
for k in trucks:
    for t in T:
        for b in charger_slots:
            for l in plugs:
                model.addCons(
                    isPlugUsed[(k, t, b, l)] <= quicksum(truckType[(k, v)] for v in electric_types),
                    name=f"only_electric_can_charge_k{k}_t{t}_b{b}_l{l}"
                )

# (11) Nicht laden während Fahrt
for k in trucks:
    for t in T:
        driving = quicksum(active[(r, t)] * routeAssign[(k, r)] for r in routes)
        model.addCons(
            quicksum(isPlugUsed[(k, t, b, l)] for b in charger_slots for l in plugs) <= 1 - driving,
            name=f"no_charge_while_driving_k{k}_t{t}"
        )

# (12) SOC max abhängig vom Typ (Diesel -> 0)
for k in trucks:
    for t in T:
        model.addCons(
            soc[(k, t)] <= quicksum(truckType[(k, v)] * truck_soc_max[v] for v in vehicle_types),
            name=f"soc_cap_by_type_k{k}_t{t}"
        )

# (13) SOC Dynamik: Laden - Verbrauch (nur E)
for k in trucks:
    for ti in range(len(T) - 1):
        t = T[ti]
        t_next = T[ti + 1]

        charged_kWh = DT_HOURS * quicksum(chargePower[(k, t, b, l)] for b in charger_slots for l in plugs)

        consumed_kWh = quicksum(
            active[(r, t)] * route_dist_per_step[r] *
            quicksum(truck_eRate[v] * routeAssignType[(k, r, v)] for v in electric_types)
            for r in routes
        )

        model.addCons(
            soc[(k, t_next)] == soc[(k, t)] + charged_kWh - consumed_kWh,
            name=f"soc_update_k{k}_t{t}"
        )

# (14) Zyklus SOC Truck (Tageswiederholung)
for k in trucks:
    model.addCons(soc[(k, T[-1])] == soc[(k, T[0])], name=f"soc_cyclic_k{k}")

# (15) Reichweitencheck am Start: wenn Truck k Route r elektrisch fährt, muss SOC reichen
for k in trucks:
    for r in routes:
        t0 = route_start_step[r]
        for v in electric_types:
            energy_need = route_dist[r] * truck_eRate[v]  # kWh
            model.addCons(
                soc[(k, t0)] >= energy_need - M_soc * (1 - routeAssignType[(k, r, v)]),
                name=f"range_check_k{k}_route{r}_type{v}"
            )

# (16) Standort-Leistungsbilanz: Ladeleistung = Netz + Discharge - Charge
for t in T:
    total_charge_load = quicksum(chargePower[(k, t, b, l)] for k in trucks for b in charger_slots for l in plugs)
    model.addCons(
        total_charge_load == gridImport[t] + storageDischarge[t] - storageCharge[t],
        name=f"site_power_balance_t{t}"
    )

# (17) Netzlimit + optional Trafo
for t in T:
    model.addCons(
        gridImport[t] <= GRID_LIMIT_KW_BASE + GRID_UPGRADE_KW * useTrafo,
        name=f"grid_limit_t{t}"
    )

# (18) Peak Definition
for t in T:
    model.addCons(gridPeak >= gridImport[t], name=f"peak_ge_import_t{t}")

# (19) Speicher: charge/discharge <= storagePower
for t in T:
    model.addCons(storageCharge[t] <= storagePower, name=f"storage_charge_le_power_t{t}")
    model.addCons(storageDischarge[t] <= storagePower, name=f"storage_discharge_le_power_t{t}")

# (20) Speicher SOC Dynamik
for ti in range(len(T) - 1):
    t = T[ti]
    t_next = T[ti + 1]
    model.addCons(
        storageSoc[t_next] == storageSoc[t] + DT_HOURS * ETA_C * storageCharge[t] - DT_HOURS * (1.0 / ETA_D) * storageDischarge[t],
        name=f"storage_soc_update_t{t}"
    )

# (21) Speicher SOC bounds inkl. DoD
for t in T:
    model.addCons(storageSoc[t] <= storageEnergy, name=f"storage_soc_le_energy_t{t}")
    model.addCons(storageSoc[t] >= MIN_SOC_FRAC * storageEnergy, name=f"storage_soc_ge_min_t{t}")

# (22) Speicher zyklisch
model.addCons(storageSoc[T[-1]] == storageSoc[T[0]], name="storage_soc_cyclic")


storage_soc_cyclic

# Zielfunktion

In [32]:
# -------------------------
# 7) Zielfunktion (€/a)
# -------------------------
fleet_cost = quicksum((truck_capex[v] + truck_opex[v]) * truckType[(k, v)] for k in trucks for v in vehicle_types)

diesel_var_cost = DAYS_PER_YEAR * quicksum(
    routeAssignType[(k, r, diesel_model)] * (
        route_dist[r] * diesel_rate_l_per_km * DIESEL_PRICE_EUR_PER_L +
        route_toll[r] * TOLL_EUR_PER_KM
    )
    for k in trucks for r in routes
)

thg_revenue = quicksum(truck_thg[v] * truckType[(k, v)] for k in trucks for v in electric_types)

charger_cost = quicksum((charger_capex[m] + charger_opex[m]) * chargerType[(b, m)] for b in charger_slots for m in charger_models)

electricity_work_kwh_per_day = quicksum(DT_HOURS * gridImport[t] for t in T)
electricity_cost = DAYS_PER_YEAR * ELECTRICITY_PRICE_EUR_PER_KWH * electricity_work_kwh_per_day + ELECTRICITY_BASE_FEE_EUR_PER_YEAR

demand_cost = DEMAND_CHARGE_EUR_PER_KW_YEAR * gridPeak
grid_upgrade_cost = GRID_UPGRADE_COST_EUR_PER_YEAR * useTrafo

storage_capex = STORAGE_CAPEX_EUR_PER_KW_YEAR * storagePower + STORAGE_CAPEX_EUR_PER_KWH_YEAR * storageEnergy
storage_cost = storage_capex + STORAGE_OPEX_FRACTION * storage_capex

total_cost = fleet_cost + diesel_var_cost - thg_revenue + charger_cost + electricity_cost + demand_cost + grid_upgrade_cost + storage_cost
model.setObjective(total_cost, "minimize")


# Modell optimieren

In [33]:
model.setRealParam("limits/time", 300) # Gib ihm 5 Minuten Zeit
model.optimize()

# Ausgabe

In [35]:
# =========================
# REPORTING im Stil deiner gewünschten Ausgabe
# =========================

def format_time_from_step(t_step: int, step_minutes: int = 15) -> str:
    minutes = t_step * step_minutes
    hh = minutes // 60
    mm = minutes % 60
    return f"{hh:02d}:{mm:02d}"

if model.getNSols() > 0:
    print("\n" + "="*50)
    print(f"ERGEBNIS: {model.getObjVal():,.2f} € Gesamtkosten/Jahr")
    print("="*50)

    # 0) Solver-Infos / Zwischenergebnis-Qualität (Gap etc.)
    try:
        print("\n--- SOLVER-STATUS ---")
        print("Status:", model.getStatus())
        print("Primal bound (beste Lösung):", model.getPrimalbound())
        print("Dual bound (beste Schranke):", model.getDualbound())
        print("MIP-Gap:", model.getGap())
        print("Solving time (s):", model.getSolvingTime())
        print("Nodes:", model.getNNodes())
    except Exception:
        pass

    # 1) ANZAHL UND TYP DER LKWS (entspricht anzahl_lkw)
    print("\n--- FLOTTENZUSAMMENSETZUNG (abgeleitet aus truckType) ---")
    for v in vehicle_types:
        anzahl = int(round(sum(model.getVal(truckType[(k, v)]) for k in trucks)))
        if anzahl > 0:
            print(f"➤ {v:12}: {anzahl} Fahrzeug(e)")

    # 2) ZUORDNUNG DER TOUREN (routeAssign und truckType)
    print("\n--- EINSATZPLAN PRO LKW-SLOT (routeAssign und truckType) ---")
    gefundene_lkws = 0
    for k in trucks:
        lkw_typ = next((v for v in vehicle_types if model.getVal(truckType[(k, v)]) > 0.5), None)
        if lkw_typ:
            tours_for_k = [r for r in routes if model.getVal(routeAssign[(k, r)]) > 0.5]
            if tours_for_k:
                gefundene_lkws += 1
                tour_liste = ", ".join(tours_for_k)
                print(f"[{lkw_typ:12}]: LKW {k:02d} fährt Tour(en) -> {tour_liste}")

    if gefundene_lkws == 0:
        print("Keine LKW-Slots mit zugewiesenen Touren gefunden (unerwartet, da jede Route genau einmal gefahren wird).")

    # 2b) DETAILED TRUCK-ROUTE-TYPE (routeAssignType)
    print("\n--- DETAILLIERTER LKW-ROUTEN-TYP PLAN (routeAssignType) ---")
    detailed_assignment_found = False
    for k in trucks:
        for r in routes:
            for v in vehicle_types:
                if model.getVal(routeAssignType[(k, r, v)]) > 0.5:
                    detailed_assignment_found = True
                    print(f"LKW {k:02d} (Typ: {v}) fährt Route {r}")
    if not detailed_assignment_found:
        print("Keine detaillierten LKW-Routen-Typ-Zuordnungen gefunden.")

    # 3) LADEINFRASTRUKTUR (entspricht anzahl_Säule)
    print("\n--- LADEINFRASTRUKTUR (abgeleitet aus chargerType) ---")
    infrastruktur_vorhanden = False
    for m in charger_models:
        anzahl_s = int(round(sum(model.getVal(chargerType[(b, m)]) for b in charger_slots)))
        if anzahl_s > 0:
            infrastruktur_vorhanden = True
            print(f"➤ {m:15}: {anzahl_s} Säule(n) beschafft")
    if not infrastruktur_vorhanden:
        print(" Keine Ladesäulen benötigt (oder E-LKW werden nicht genutzt / laden nicht).")

    # 3b) ZUORDNUNG SÄULENBESTAND -> TYP (bei uns direkt chargerType)
    print("\n--- ZUORDNUNG SÄULEN-SLOTS ZU MODELLEN (chargerType) ---")
    pole_type_assignment_found = False
    for b in charger_slots:
        for m in charger_models:
            if model.getVal(chargerType[(b, m)]) > 0.5:
                pole_type_assignment_found = True
                print(f"Säule/Slot {b} ist vom Typ: {m}")
    if not pole_type_assignment_found:
        print("Keine Säulen-Slots zugewiesen.")

    # 4) DETAILLIERTE LADEAKTIVITÄTEN UND ENERGIEFLÜSSE
    print("\n--- DETAILLIERTE LADEAKTIVITÄTEN UND ENERGIEFLÜSSE (isPlugUsed, chargePower) ---")
    charging_activity_found_overall = False

    for t in T:
        time_has_activity = False
        for k in trucks:
            for b in charger_slots:
                for l in plugs:
                    if model.getVal(isPlugUsed[(k, t, b, l)]) > 0.5:
                        charging_activity_found_overall = True
                        time_has_activity = True

                        lkw_typ = next((v for v in vehicle_types if model.getVal(truckType[(k, v)]) > 0.5), "N/A")
                        p_kw = model.getVal(chargePower[(k, t, b, l)])

                        # Optional: "Plug liefert" (in deinem Output gab es eine Ladepunkt-Variable)
                        # Bei uns gibt es keine eigene Variable pro Plug-Limit. Wir geben stattdessen:
                        # - Gesamtleistung am Charger-Slot b (über beide Plugs)
                        charger_total_kw = sum(model.getVal(chargePower[(kk, t, b, ll)]) for kk in trucks for ll in plugs)

                        uhrzeit = format_time_from_step(t, STEP_MINUTES)
                        print(
                            f"[{uhrzeit}] LKW {k:02d} (Typ: {lkw_typ}) lädt an Säule {b}, Ladepunkt {l} "
                            f"mit {p_kw:.1f} kW (Gesamt an Säule {b}: {charger_total_kw:.1f} kW)."
                        )

        if time_has_activity:
            print("-" * 100)

    if not charging_activity_found_overall:
        print("Keine Ladeaktivitäten oder Energieflüsse registriert.")

    # 5) ZEITLICHER VERLAUF: Aktivität & SOC
    print("\n--- ZEITLICHER VERLAUF DER LKW-AKTIVITÄT & BATTERIESTAND (soc) ---")
    print("Diese detaillierte Ansicht zeigt den Status jedes LKWs pro Zeitintervall.")
    print("\n" + "="*80)
    print(f"{'Uhrzeit':<8} | {'LKW':<8} | {'Typ':<12} | {'Aktivität':<26} | {'Batterie'}")
    print("-" * 80)

    for k in trucks:
        lkw_typ = next((v for v in vehicle_types if model.getVal(truckType[(k, v)]) > 0.5), None)
        if not lkw_typ:
            continue

        last_status_display = None
        last_soc_display = None

        for t in T:
            uhrzeit = format_time_from_step(t, STEP_MINUTES)
            soc_val = model.getVal(soc[(k, t)])

            # Status bestimmen: fährt eine Route?
            current_status = "Standby"
            for r in routes:
                if model.getVal(routeAssign[(k, r)]) > 0.5 and active[(r, t)] > 0.5:
                    current_status = f"🚚 Tour {r}"
                    break

            # Wenn nicht fährt: lädt er?
            if current_status == "Standby":
                for b in charger_slots:
                    for l in plugs:
                        if model.getVal(isPlugUsed[(k, t, b, l)]) > 0.5:
                            current_status = f"⚡ Laden an S.{b}/LP.{l}"
                            break
                    if current_status != "Standby":
                        break

            # Nur drucken, wenn Status wechselt oder SOC sich merklich ändert
            if (current_status != last_status_display) or (last_soc_display is not None and abs(soc_val - last_soc_display) > 0.1):
                b_anzeige = f"{soc_val:6.1f} kWh" if lkw_typ != diesel_model else "---"
                print(f"{uhrzeit:<8} | LKW {k:02d} | {lkw_typ:<12} | {current_status:<26} | {b_anzeige}")
                last_status_display = current_status
                last_soc_display = soc_val

        print("-" * 80)

else:
    print("Keine Lösung gefunden.")


ERGEBNIS: 1,081,248.80 € Gesamtkosten/Jahr

--- SOLVER-STATUS ---
Status: timelimit
Primal bound (beste Lösung): 1081248.8
Dual bound (beste Schranke): 1000.0
MIP-Gap: 1080.2488
Solving time (s): 300.010477
Nodes: 1

--- FLOTTENZUSAMMENSETZUNG (abgeleitet aus truckType) ---
➤ ActrosL     : 14 Fahrzeug(e)
➤ eActros600  : 1 Fahrzeug(e)

--- EINSATZPLAN PRO LKW-SLOT (routeAssign und truckType) ---
[ActrosL     ]: LKW 07 fährt Tour(en) -> t-4
[ActrosL     ]: LKW 08 fährt Tour(en) -> t-5
[ActrosL     ]: LKW 09 fährt Tour(en) -> t-6
[ActrosL     ]: LKW 10 fährt Tour(en) -> s-1
[ActrosL     ]: LKW 11 fährt Tour(en) -> s-2
[ActrosL     ]: LKW 12 fährt Tour(en) -> s-3
[ActrosL     ]: LKW 13 fährt Tour(en) -> s-4
[ActrosL     ]: LKW 14 fährt Tour(en) -> w1, r2
[ActrosL     ]: LKW 15 fährt Tour(en) -> w2
[ActrosL     ]: LKW 16 fährt Tour(en) -> w3, r1
[ActrosL     ]: LKW 17 fährt Tour(en) -> w4, k1
[ActrosL     ]: LKW 18 fährt Tour(en) -> w5, r3
[ActrosL     ]: LKW 19 fährt Tour(en) -> w6, h3
[A